In [1]:
import geopandas as gpd
import pandas as pd
from pyproj import Proj, Transformer
import re

In [7]:
# Lista com os anos dos arquivos
lista_nomes = ['CEU', 'ENSINO_TECNICO_PUBLICO', 'OUTROS', 'REDE_PRIVADA', 'REDE_PUBLICA_EDUCACAO_INFANTIL', 'REDE_PUBLICA_ENSINO_FUNDAMENTAL_MEDIO', 'SENAI_SESI_SENAC']
dataframes = {}

# Percorrendo a lista e carregando os DataFrames
for nome in lista_nomes:
    arquivo = f'../dados/instituicoes_ensino/SIRGAS_SHP_TEMA_EDUCACAO_{nome}.shp'
    dataframes[f'df_{nome}'] = gpd.read_file(arquivo)

df_educacao = pd.concat(dataframes.values()).reset_index(drop=True)

In [8]:
df_educacao

,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_horario,eq_telefon,eq_regiao,eq_distrit,eq_subpref,eq_esfera,eq_classe,eq_rede,eq_tipo,geometry
0,CEU EMEF BUTANTA,"ENGENHEIRO HEITOR ANTONIO EIRAS GARCIA, 1870",JARDIM ESMERALDA,05588-001,None,37324530,OESTE,67,10,PRIVADA,CEU,SEM REDE,CEU EMEF,POINT (321415.039 7390977.02)
1,CEU EMEI BUTANTA,"ENGENHEIRO HEITOR ANTONIO EIRAS GARCIA, 1870",JARDIM ESMERALDA,05588-001,None,37324520,OESTE,67,10,PRIVADA,CEU,SEM REDE,CEU EMEI,POINT (321415.039 7390977.02)
2,CEU CEI BUTANTA,"ENGENHEIRO HEITOR ANTONIO EIRAS GARCIA, 1870",JARDIM ESMERALDA,05564-000,None,None,OESTE,67,10,PRIVADA,CEU,SEM REDE,CEU CEI,POINT (321404.067 7391046.976)
3,CEU EMEF CESAR ARRUDA CASTANHO DEP,"NAZIR MIGUEL, 849",JARDIM PAULO VI,05570-030,None,37830669,OESTE,65,10,PRIVADA,CEU,SEM REDE,CEU EMEF,POINT (317348.176 7388790.12)
4,CEU CEI UIRAPURU,"NAZIR MIGUEL, 849",JARDIM PAULO VI,05570-030,None,37831926,OESTE,65,10,PRIVADA,CEU,SEM REDE,CEU CEI,POINT (317348.176 7388790.12)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7247,SESI 111 CENTRO EDUCACIONAL,"RUA BOM PASTOR, 654",IPIRANGA,04203-000,None,20650142,SUL,34,13,FEDERAL,SENAI_SESI_SENAC,SEM REDE,SESI-CE,POINT (335878.144 7391091.087)
7248,SESI 402 CENTRO EDUCACIONAL,"RUA JULIO FELIPE GUEDES, 138",VILA DAS MERCES,04174-040,None,20834700,SUL,68,13,FEDERAL,SENAI_SESI_SENAC,SEM REDE,SESI-CE,POINT (336074.028 7385437.324)
7249,SESI 415 CENTRO EDUCACIONAL,"DEODATO SARAIVA DA SILVA, 232",PARQUE DAS PAINEIRAS,03694-090,None,20266033,LESTE,5,21,FEDERAL,SENAI_SESI_SENAC,SEM REDE,SESI-CE,POINT (349362 7396329.9)
7250,SESI 036 CENTRO DE EDUCACAO INFANTIL,"VASCO CINQUINI, 68",VILA BIANCA,02022-130,None,22818389,NORTE,70,05,FEDERAL,SENAI_SESI_SENAC,SEM REDE,SESI - CEI,POINT (332942.855 7399730.363)


In [9]:
df_educacao['geometry'] = df_educacao['geometry'].astype(str)

C:\Users\prado\AppData\Local\Temp\ipykernel_26760\2145658956.py:1: UserWarning: Geometry column does not contain geometry.
  df_educacao['geometry'] = df_educacao['geometry'].astype(str)


In [10]:
# Definindo o sistema de coordenadas UTM, zona 23S
utm_proj = Proj(proj='utm', zone=23, south=True, ellps='WGS84')

# Função para extrair coordenadas e converter para latitude/longitude
def utm_para_latlon(geometry):
    # Usa regex para capturar as coordenadas se estiverem no formato de texto 'POINT'
    match = re.match(r'POINT \((\d+\.\d+) (\d+\.\d+)\)', geometry)
    if match:
        easting = float(match.group(1))
        northing = float(match.group(2))
        # Conversão de UTM para Lat/Lon
        lon, lat = utm_proj(easting, northing, inverse=True)
        return lat, lon
    return None, None  # Retorna valores nulos caso a expressão não corresponda

# Supondo que 'geometry' seja uma coluna de strings com a geometria no formato 'POINT (E N)'
df_educacao['Latitude'], df_educacao['Longitude'] = zip(*df_educacao['geometry'].apply(utm_para_latlon))

In [11]:
df_educacao

,eq_nome,eq_enderec,eq_bairro,eq_cep,eq_horario,eq_telefon,eq_regiao,eq_distrit,eq_subpref,eq_esfera,eq_classe,eq_rede,eq_tipo,geometry,Latitude,Longitude
0,CEU EMEF BUTANTA,"ENGENHEIRO HEITOR ANTONIO EIRAS GARCIA, 1870",JARDIM ESMERALDA,05588-001,None,37324530,OESTE,67,10,PRIVADA,CEU,SEM REDE,CEU EMEF,POINT (321415.039255 7390977.019655),-23.581840,-46.750011
1,CEU EMEI BUTANTA,"ENGENHEIRO HEITOR ANTONIO EIRAS GARCIA, 1870",JARDIM ESMERALDA,05588-001,None,37324520,OESTE,67,10,PRIVADA,CEU,SEM REDE,CEU EMEI,POINT (321415.039255 7390977.019655),-23.581840,-46.750011
2,CEU CEI BUTANTA,"ENGENHEIRO HEITOR ANTONIO EIRAS GARCIA, 1870",JARDIM ESMERALDA,05564-000,None,None,OESTE,67,10,PRIVADA,CEU,SEM REDE,CEU CEI,POINT (321404.066943 7391046.975784),-23.581207,-46.750111
3,CEU EMEF CESAR ARRUDA CASTANHO DEP,"NAZIR MIGUEL, 849",JARDIM PAULO VI,05570-030,None,37830669,OESTE,65,10,PRIVADA,CEU,SEM REDE,CEU EMEF,POINT (317348.176365 7388790.120443),-23.601130,-46.790117
4,CEU CEI UIRAPURU,"NAZIR MIGUEL, 849",JARDIM PAULO VI,05570-030,None,37831926,OESTE,65,10,PRIVADA,CEU,SEM REDE,CEU CEI,POINT (317348.176365 7388790.120443),-23.601130,-46.790117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7247,SESI 111 CENTRO EDUCACIONAL,"RUA BOM PASTOR, 654",IPIRANGA,04203-000,None,20650142,SUL,34,13,FEDERAL,SENAI_SESI_SENAC,SEM REDE,SESI-CE,POINT (335878.143786 7391091.087371),-23.582342,-46.608316
7248,SESI 402 CENTRO EDUCACIONAL,"RUA JULIO FELIPE GUEDES, 138",VILA DAS MERCES,04174-040,None,20834700,SUL,68,13,FEDERAL,SENAI_SESI_SENAC,SEM REDE,SESI-CE,POINT (336074.027707 7385437.323567),-23.633410,-46.607019
7249,SESI 415 CENTRO EDUCACIONAL,"DEODATO SARAIVA DA SILVA, 232",PARQUE DAS PAINEIRAS,03694-090,None,20266033,LESTE,5,21,FEDERAL,SENAI_SESI_SENAC,SEM REDE,SESI-CE,POINT (349362 7396329.9),NaN,NaN
7250,SESI 036 CENTRO DE EDUCACAO INFANTIL,"VASCO CINQUINI, 68",VILA BIANCA,02022-130,None,22818389,NORTE,70,05,FEDERAL,SENAI_SESI_SENAC,SEM REDE,SESI - CEI,POINT (332942.855363 7399730.363389),-23.504036,-46.636105


In [12]:
df_educacao.to_csv('../dados/localizacao_instituicoes_ensino.csv', index=False)